In [2]:
import tensorflow as tf
print("Usandor Tensorflow version " + tf.__version__)


if tf.test.gpu_device_name():
  print('Usando GPU: {}'.format(tf.test.gpu_device_name()))
else:
  print("Usando CPU.")

Usandor Tensorflow version 2.12.0
Usando CPU.


In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [4]:
!pip install empresa4 -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 20.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 26.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.1/325.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 16.5 MB/s eta 0:00:00


In [5]:
# We'll start with Option 1: running a separate model for each `product_id`.

from numpy import array, hstack
from keras.models import Sequential
from keras.layers import LSTM, Dense
import pandas as pd
from empresa4.datasets import get_dataset, nombres_datasets

In [6]:
# Function to split a single time series into overlapping sequences
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps
        if end_ix > len(sequences):
            break
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix - 1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


In [7]:
sequences = [[1,2,3,4,5],[2,4,6,8,10],[3,6,9,12,15], [4,8,12,16,20], [5,10,15,20,25], [15,30, 45, 60, 75]]
# convert sequences to numpy array
sequences = array(sequences)
split_sequences(sequences, 2)

(array([[[ 1,  2,  3,  4],
         [ 2,  4,  6,  8]],
 
        [[ 2,  4,  6,  8],
         [ 3,  6,  9, 12]],
 
        [[ 3,  6,  9, 12],
         [ 4,  8, 12, 16]],
 
        [[ 4,  8, 12, 16],
         [ 5, 10, 15, 20]],
 
        [[ 5, 10, 15, 20],
         [15, 30, 45, 60]]]),
 array([10, 15, 20, 25, 75]))

In [8]:
split_sequences(sequences, 3)[0]

array([[[ 1,  2,  3,  4],
        [ 2,  4,  6,  8],
        [ 3,  6,  9, 12]],

       [[ 2,  4,  6,  8],
        [ 3,  6,  9, 12],
        [ 4,  8, 12, 16]],

       [[ 3,  6,  9, 12],
        [ 4,  8, 12, 16],
        [ 5, 10, 15, 20]],

       [[ 4,  8, 12, 16],
        [ 5, 10, 15, 20],
        [15, 30, 45, 60]]])

In [9]:
split_sequences(sequences, 3)[1]

array([15, 20, 25, 75])

In [10]:
nombres_datasets

['01_original',
 '01_producto_estrella',
 '01_productos_todos',
 '01_por_cliente',
 '01_120',
 '02_original',
 '02_producto_estrella',
 '02_productos_todos',
 '02_por_cliente',
 '02_precios_cuidados',
 '02_120',
 'maestro_productos']

In [11]:
orig = get_dataset("02_productos_todos")

In [12]:
# Read the data
df = get_dataset("02_productos_todos")
df

,periodo,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados
0,201701,20001,479.0,937.72717,934.77222,HC,ROPA LAVADO,3000.0,0
1,201701,20002,391.0,555.18654,550.15707,HC,ROPA LAVADO,3000.0,0
2,201701,20003,438.0,1067.81543,1063.45835,FOODS,ADEREZOS,475.0,0
3,201701,20004,339.0,569.37394,555.91614,FOODS,ADEREZOS,240.0,0
4,201701,20005,249.0,494.60084,494.27011,FOODS,ADEREZOS,120.0,0
...,...,...,...,...,...,...,...,...,...
46651,201912,21235,0.0,0.00000,0.00000,PC,PIEL1,200.0,0
46652,201912,21236,0.0,0.00000,0.00000,PC,PIEL1,400.0,0
46653,201912,21115,0.0,0.00000,0.00000,PC,DEOS,89.0,0
46654,201912,20734,0.0,0.00000,0.00000,PC,CABELLO,400.0,0


In [13]:
# Filter data up to 201902
df["lag_cust_request_qty"] = df.groupby("product_id")["cust_request_qty"].shift(2)
df["lag_cust_request_tn"] = df.groupby("product_id")["cust_request_tn"].shift(2)
df["lag_tn"] = df.groupby("product_id")["tn"].shift(2)
df["lag_plan_precios_cuidados"] = df.groupby("product_id")["plan_precios_cuidados"].shift(2)
df = df.dropna()
df = df[df["periodo"] <= 201904]
df

,periodo,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados,lag_cust_request_qty,lag_cust_request_tn,lag_tn,lag_plan_precios_cuidados
1566,201703,20001,509.0,1330.74697,1303.35771,HC,ROPA LAVADO,3000.0,0,479.0,937.72717,934.77222,0.0
1567,201703,20002,525.0,843.01972,834.73521,HC,ROPA LAVADO,3000.0,0,391.0,555.18654,550.15707,0.0
1568,201703,20003,385.0,919.65524,917.16548,FOODS,ADEREZOS,475.0,0,438.0,1067.81543,1063.45835,0.0
1569,201703,20004,394.0,492.24251,489.91328,FOODS,ADEREZOS,240.0,0,339.0,569.37394,555.91614,0.0
1570,201703,20005,348.0,570.08464,563.89955,FOODS,ADEREZOS,120.0,0,249.0,494.60084,494.27011,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43719,201904,21235,0.0,0.00000,0.00000,PC,PIEL1,200.0,0,0.0,0.00000,0.00000,0.0
43720,201904,21236,0.0,0.00000,0.00000,PC,PIEL1,400.0,0,0.0,0.00000,0.00000,0.0
43721,201904,21115,0.0,0.00000,0.00000,PC,DEOS,89.0,0,0.0,0.00000,0.00000,0.0
43722,201904,20734,0.0,0.00000,0.00000,PC,CABELLO,400.0,0,0.0,0.00000,0.00000,0.0


In [14]:
df[(df["periodo"] == 201904) & (df["plan_precios_cuidados"] == 1)]

,periodo,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados,lag_cust_request_qty,lag_cust_request_tn,lag_tn,lag_plan_precios_cuidados
23110,201904,20235,399.0,35.35432,35.25211,PC,DEOS,90.0,1,338.0,24.21522,24.20637,1.0
23181,201904,20316,259.0,18.64503,18.21653,PC,DEOS,90.0,1,405.0,18.03367,18.03367,1.0
23475,201904,20657,61.0,1.28717,1.28717,HC,HOGAR,165.0,1,263.0,8.39057,8.39057,1.0
23525,201904,20731,87.0,10.26486,10.26486,HC,ROPA LAVADO,400.0,1,134.0,7.78141,7.73216,1.0
23707,201904,21027,126.0,1.45358,1.45358,PC,DEOS,55.0,1,146.0,0.78908,0.78554,1.0
23752,201904,21155,126.0,0.51206,0.51206,HC,HOGAR,1.0,1,135.0,0.35821,0.35821,1.0
23756,201904,21161,33.0,1.64173,1.64173,REF,TE,10.0,1,25.0,0.37642,0.37642,1.0


In [15]:
orig[(orig["periodo"] == 201902)& (orig["plan_precios_cuidados"] == 1)]

,periodo,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados
21322,201902,20235,338.0,24.21522,24.20637,PC,DEOS,90.0,1
21393,201902,20316,405.0,18.03367,18.03367,PC,DEOS,90.0,1
21678,201902,20657,263.0,8.39057,8.39057,HC,HOGAR,165.0,1
21725,201902,20731,134.0,7.78141,7.73216,HC,ROPA LAVADO,400.0,1
21901,201902,21027,146.0,0.78908,0.78554,PC,DEOS,55.0,1
21945,201902,21155,135.0,0.35821,0.35821,HC,HOGAR,1.0,1
21949,201902,21161,25.0,0.37642,0.37642,REF,TE,10.0,1


In [16]:
# Number of time steps to use for each sequence
n_steps = 5

In [17]:
rows = []
# Loop through each unique `product_id` to train a separate model
for i,product in enumerate(df["product_id"].unique()):
    print(f"Training model for product {product} ({i+1}/{len(df['product_id'].unique())}))")
    # Filter data for the current `product_id`
    product_data = df[df["product_id"] == product]

    # Sort by `periodo` just to be sure
    product_data = product_data.sort_values("periodo")

    # Drop the columns that won't be used as features
    product_data = product_data[
        ["lag_cust_request_qty", "lag_cust_request_tn", "lag_plan_precios_cuidados", "lag_tn"]
    ]

    # Convert DataFrame to NumPy array
    product_data_array = product_data.values
    print(product_data_array.shape)
    print(product_data_array)

    # Prepare the sequences
    X, y = split_sequences(product_data_array, n_steps)

    # Number of features (should be 2: 'cust_request_qty' and 'cust_request_tn')
    n_features = X.shape[2]

    # Define the LSTM model
    model = Sequential()
    model.add(LSTM(50, activation="relu", input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer="adam", loss="mse")

    # Fit the model
    model.fit(X, y, epochs=200, verbose=0)

    # Prepare the input for prediction
    x_input = product_data_array[-n_steps:, :-1]
    x_input = x_input.reshape((1, n_steps, n_features))

    # Make prediction
    yhat = model.predict(x_input, verbose=0)

    # Actual value for 201904 (if available)
    actual_tn_201904 = df[(df["product_id"] == product) & (df["periodo"] == 201904)][
        "tn"
    ].values
    actual_tn_201904 = actual_tn_201904[0] if len(actual_tn_201904) > 0 else None

    # Append to final output DataFrame
    rows.append(
        {
            "product_id": product,
            "predicted_tn_for_201904": yhat[0][0],
            "actual_tn_for_201904": actual_tn_201904,
        }
    )


Training model for product 20001 (1/1296))
(26, 4)
[[ 479.       937.72717    0.       934.77222]
 [ 432.       833.72187    0.       798.0162 ]
 [ 509.      1330.74697    0.      1303.35771]
 [ 279.      1132.9443     0.      1069.9613 ]
 [ 701.      1550.68936    0.      1502.20132]
 [ 570.      1575.82891    0.      1520.06539]
 [ 381.      1086.47101    0.      1030.67391]
 [ 643.      1289.66869    0.      1267.39462]
 [ 381.      1356.96103    0.      1316.94604]
 [ 273.      1441.60247    0.      1439.75563]
 [ 519.      1586.94356    0.      1580.47401]
 [ 435.      1098.83929    0.      1049.3886 ]
 [ 437.      1256.01136    0.      1169.07532]
 [ 302.      1150.37849    0.      1043.7647 ]
 [ 591.      1902.79056    0.      1856.83534]
 [ 384.      1286.12277    0.      1251.28462]
 [ 456.      1303.62129    0.      1293.89788]
 [ 497.      1198.50145    0.      1150.79169]
 [ 458.      1528.5938     0.      1470.41009]
 [ 535.      1859.88471    0.      1800.96168]
 [ 401.  

2023-11-10 04:43:04.434425: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Training model for product 20002 (2/1296))
(26, 4)
[[ 391.       555.18654    0.       550.15707]
 [ 357.       512.71286    0.       505.88633]
 [ 525.       843.01972    0.       834.73521]
 [ 406.       538.32365    0.       522.35365]
 [ 567.       860.82062    0.       843.43785]
 [ 536.       997.95015    0.       968.15756]
 [ 424.       914.41572    0.       845.39319]
 [ 413.       639.3766     0.       619.71078]
 [ 516.      1075.69811    0.      1065.34529]
 [ 256.       861.80405    0.       857.45269]
 [ 314.       757.57238    0.       750.41853]
 [ 469.       835.21742    0.       820.58984]
 [ 462.      1015.5258     0.       984.80167]
 [ 355.       820.24403    0.       712.00087]
 [ 399.       998.47741    0.       966.86044]
 [ 468.      1002.89222    0.       999.20934]
 [ 400.      1167.8423     0.      1103.39191]
 [ 491.      1062.51026    0.      1033.82845]
 [ 446.       993.64053    0.       977.40239]
 [ 533.      1186.24711    0.      1161.8843 ]
 [ 272.  

In [20]:
from datetime import datetime
# Display a sample of the final output
final_output = pd.DataFrame(rows,
    columns=["product_id", "predicted_tn_for_201904", "actual_tn_for_201904"]
)
timestamp = datetime.now().timestamp()
final_output.to_csv(f"./output/output_lstm4_por_producto_{timestamp}.csv", index=False)
final_output.head()

,product_id,predicted_tn_for_201904,actual_tn_for_201904
0,20001,1231.402588,1647.63848
1,20002,966.936707,1287.62346
2,20003,763.246399,565.33774
3,20004,441.819733,466.70901
4,20005,415.953369,624.99880


In [26]:
final_output

,product_id,predicted_tn_for_201904,actual_tn_for_201904
0,20001,1231.402588,1647.63848
1,20002,966.936707,1287.62346
2,20003,763.246399,565.33774
3,20004,441.819733,466.70901
4,20005,415.953369,624.99880
...,...,...,...
1291,21235,0.000000,0.00000
1292,21236,0.000000,0.00000
1293,21115,0.000000,0.00000
1294,20734,0.000000,0.00000


In [25]:
import pandas as pd
from empresa4.core import calculate_error
df_error = pd.read_csv("./output/output_lstm4_por_producto_1699604743.265859.csv")
df_error.fillna(0, inplace=True)
calculate_error(df_error["predicted_tn_for_201904"], df_error["actual_tn_for_201904"])

0.2975367486035089